# Theory

In general is a way to reduce model overfitting and variance. Uses penslty haperparameters

L1 - LASSO regression: Limits the size of the coefficients, by adding penalty equal to the absolute value of the magnitute of coeeficients.  May make some coeficients = 0 to decide which coeficients to consider

L2 - Ridge regression: All coeficients are shrunk by the same factor, by adding a penalty equal to the square of the magnitude of coefficients.   Does not elliminate coefficients

L1 and L2 - Elastic net: Combines both models with a certain ratio between them ((1-A) *L1 + A(L2))

@

hold out set

train - validation - test

# Data Set Up

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [42]:
df = pd.read_csv("/Users/vazgenosipov/Desktop/Python/Python Data analys/UNZIP_FOR_NOTEBOOKS_FINAL/08-Linear-Regression-Models/Advertising.csv")


In [43]:
X = df.drop("sales", axis=1)

# will create an X variable with no Y variable in the columns

In [44]:
y = df["sales"]

# create y variable

In [45]:
from sklearn.preprocessing import PolynomialFeatures

In [46]:
# create polynomical converter

polynomial_converter = PolynomialFeatures(degree=3, include_bias=False)

In [47]:
# convert the data into polynomial

poly_features = polynomial_converter.fit_transform(X)

In [48]:
# check the shape

print (X.shape, "X")
print (poly_features.shape, "poly_features")

(200, 3) X
(200, 19) poly_features


In [49]:
from sklearn.model_selection import train_test_split

In [50]:
# import the train test split model
# import poly feeatured data

X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=42)

In [51]:
from sklearn.preprocessing import StandardScaler

In [52]:
# create an instance scaler

scaler = StandardScaler()

In [53]:
# fit the data onto training set so that the model what know std, mean and other data 
# from the test set which it will later estimate (this is called information leaking if run on whole set)

# you dont fit y variable 

scaler.fit(X_train)

StandardScaler()

In [54]:
# onced it is fitted than transform the data


X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# L2 - Ridge Regression

In [55]:
from sklearn.linear_model import Ridge

In [56]:
# create a ridge model
# that will penalise the higher steps in the slope of the best fit line
# introducing some additional bies that will decrease the model performance on the train data 
# but will be general enough to boost the performance on unseen data

# due to some of the sklearn mechanics, alha = lamda to keep everything in uniform form 
# also all models evaluet the performance based on the higher scores
# even MSE this is why they are negative, so higher numbers are performing better

ridge_model = Ridge(alpha=10)

In [57]:
# use just like normal linear regression

ridge_model.fit(X_train, y_train)

Ridge(alpha=10)

In [58]:
test_predictions = ridge_model.predict(X_test)

In [59]:
# to get performance results 

from sklearn.metrics import mean_absolute_error, mean_squared_error

In [60]:
MAE = mean_absolute_error(y_test, test_predictions)

In [61]:
MAE

0.6296591346758601

In [62]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [63]:
RMSE

0.8916327541710889

In [64]:
from sklearn.linear_model import RidgeCV

# will cross validate on rigde

In [65]:
# will perfrom the cross validation on variaty alpha values and will return the best parameter
# if you leave cv (number of folds in cross vall) it will use leave one out
# basically performning the train on all the data but one to test
# and repeat this prodicure by the number of row in the data - very enery consuming 

ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0))

In [66]:
# we use x_test and y_test as a hold out for testing so that our model
# is unfamiliar with it
# since it uses cv = k number of folds to validate the data from the X_train portion of the data 

ridge_cv_model.fit(X_train, y_train)

RidgeCV()

In [67]:
# which alpha (lambda) performed the best

ridge_cv_model.alpha_

0.1

In [69]:
# lest use different scoring method (use different method of tracking error)

from sklearn.metrics import SCORERS

In [76]:
# this a dictionary with different performance evaluation matrix 
# take a look that model uses the highest scores to evaluate a better alpha
# but some cant be high as MAS then we have neg_mean_abs to work for it

# use to look up the scores

SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'matthews_corrcoef', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_s

In [97]:
# lets use the different scoring metric for evaluating the best parameter


ridge_cv_model = RidgeCV(alphas=(1,2,3,15,0.1,0.001, 100), scoring='neg_mean_squared_error')

In [98]:
ridge_cv_model.fit(X_train, y_train)

RidgeCV(alphas=(1, 2, 3, 15, 0.1, 0.001, 100), scoring='neg_mean_squared_error')

In [99]:
ridge_cv_model.alpha_

# still the best performing alpha

0.1

In [100]:
# now lest compare the results with new alpha on unseen one hold out data


test_predictions_cv = ridge_cv_model.predict(X_test)

In [104]:
# we will compare it ot the data where alpha = 10

# in the model we use hold out set to compare the results

MAE = mean_absolute_error(y_test, test_predictions_cv)

In [102]:
MAE

0.46671241131149477

In [103]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions_cv))
RMSE

0.5945136671792505

In [ ]:
# as you see the test results for the error data are lower now
# which indicates the increase in performance of the matrix

In [105]:
ridge_cv_model.coef_

array([ 5.90523815,  0.46316396,  0.68028713, -6.17743395,  3.73671928,
       -1.40708382,  0.00624704,  0.11128917, -0.2617823 ,  2.17135744,
       -0.51480159,  0.70587211,  0.60311504, -0.53271216,  0.5716495 ,
       -0.34685826,  0.36744388, -0.03938079, -0.12192939])

# L1 - Lasso Regression

In [133]:
from sklearn.linear_model import LassoCV

In [134]:
lasso_cv_model = LassoCV(eps=0.1, n_alphas=100, cv=5)

# you give the model the number of alphas to check
# eps is the min and max value (min/max)
# the smaller the eps the larger subset of values you check

# depending in the size of the difference in eps and n_aphas 
# we may increase the number of iterations by max_iter


# the formulas is absolute value 

In [135]:
lasso_cv_model.fit(X_train, y_train)

LassoCV(cv=5, eps=0.1)

In [136]:
lasso_cv_model.alpha_

0.4924531806474872

In [137]:
test_predictions = lasso_cv_model.predict(X_test)

In [138]:
MAE = mean_absolute_error(y_test, test_predictions)

In [139]:
MAE

0.6811456342837978

In [140]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [141]:
RMSE

1.0349127365478723

In [142]:
lasso_cv_model.coef_

# it only considers several of the feature establishing the other 
# not significant

# this is the benefit of lasso its results are not as accurate 
# but it uses a more simple model and will produce the output faster

array([0.97675148, 0.        , 0.        , 0.        , 3.8148913 ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

# L1 and L2 - Elastic Net

In [124]:
from sklearn.linear_model import ElasticNetCV

In [127]:
elastic_model = ElasticNetCV(l1_ratio=[.1,0.5,.7,.9,.95,.99,1],
                            eps=0.001, n_alphas=100, max_iter=1000000)


# l1_ratio uses more lasso than redge, also recomended to use more lasso for better accuracy

In [129]:
elastic_model.fit(X_train, y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=1000000)

In [130]:
# best performing l1 ratio
# the result 1 means that lasso is the best option here
# no ridge was used


elastic_model.l1_ratio_

1.0

In [131]:
elastic_model.alpha_

# the exact same alpha from before
# we expanded the eterations and increase the difference in eps 
# that why re see the difference in 00ss

0.0049245318064748715

In [144]:
test_predictions = elastic_model.predict(X_test)

In [145]:
MAE = mean_absolute_error(y_test, test_predictions)
MAE


0.5123045552899819

In [146]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))
MAE

0.5123045552899819